In [ ]:
!unzip /content/classical-piano-type0.zip

Archive:  /content/classical-piano-type0.zip
   creating: content/Keras-LSTM-Music-Generator/classical-piano-type0/
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/beethoven_opus22_3.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/ty_maerz.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_march.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_wanderer.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/ty_januar.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/chpn-p18.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_elfentanz.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/brahms_opus1_2.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/schumm-3.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano-type0/schu_143_2.mid  

In [ ]:
!unzip /content/classical-piano.zip

Archive:  /content/classical-piano.zip
   creating: content/Keras-LSTM-Music-Generator/classical-piano/
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/beethoven_opus22_3.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/ty_maerz.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/grieg_march.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/grieg_wanderer.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/ty_januar.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/chpn-p18.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/grieg_elfentanz.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/brahms_opus1_2.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/schumm-3.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/schu_143_2.mid  
  inflating: content/Keras-LSTM-Music-Generator/classical-piano/beethov

In [ ]:
def calculate_ppl_from_training_log():
    """ Đọc log training và tính perplexity trên tập validation """
    try:
        # Đọc file CSV log
        df = pd.read_csv('training_log.csv')
        # Tìm loss validation nhỏ nhất
        best_val_loss = df['val_loss'].min()
        # Tính Perplexity (PPL)
        ppl = math.exp(best_val_loss)
        print(f"Perplexity (PPL) trên tập validation: {ppl:.4f}")
        return ppl
    except Exception as e:
        print(f"Lỗi khi đọc log training: {e}")
        return None

# Gọi hàm để tính PPL
ppl_val = calculate_ppl_from_training_log()


In [ ]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from keras import Model
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, concatenate
from sklearn.model_selection import train_test_split

def train_network():
    """ Train a Neural Network to generate music """
    notes, offsets, durations = get_notes()

    # Prepare notes
    n_vocab_notes = len(set(notes))
    network_input_notes, network_output_notes = prepare_sequences(notes, n_vocab_notes)

    # Prepare offsets
    n_vocab_offsets = len(set(offsets))
    network_input_offsets, network_output_offsets = prepare_sequences(offsets, n_vocab_offsets)

    # Prepare durations
    n_vocab_durations = len(set(durations))
    network_input_durations, network_output_durations = prepare_sequences(durations, n_vocab_durations)

    # Split data into training, validation, and test sets
    X_notes_train, X_notes_test, y_notes_train, y_notes_test = train_test_split(network_input_notes, network_output_notes, test_size=0.2, random_state=42)
    X_offsets_train, X_offsets_test, y_offsets_train, y_offsets_test = train_test_split(network_input_offsets, network_output_offsets, test_size=0.2, random_state=42)
    X_durations_train, X_durations_test, y_durations_train, y_durations_test = train_test_split(network_input_durations, network_output_durations, test_size=0.2, random_state=42)

    # Further split training into training and validation sets
    X_notes_train, X_notes_val, y_notes_train, y_notes_val = train_test_split(X_notes_train, y_notes_train, test_size=0.1, random_state=42)
    X_offsets_train, X_offsets_val, y_offsets_train, y_offsets_val = train_test_split(X_offsets_train, y_offsets_train, test_size=0.1, random_state=42)
    X_durations_train, X_durations_val, y_durations_train, y_durations_val = train_test_split(X_durations_train, y_durations_train, test_size=0.1, random_state=42)

    # Create the model
    model = create_network(X_notes_train, n_vocab_notes, X_offsets_train, n_vocab_offsets, X_durations_train, n_vocab_durations)

    # Callbacks to save the best model and track training/validation loss
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.keras"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    csv_logger = CSVLogger('training_log.csv', append=True)

    callbacks_list = [checkpoint, early_stop, csv_logger]

    # Train the model
    model.fit([X_notes_train, X_offsets_train, X_durations_train], [y_notes_train, y_offsets_train, y_durations_train],
              epochs=100, batch_size=64, validation_data=([X_notes_val, X_offsets_val, X_durations_val], [y_notes_val, y_offsets_val, y_durations_val]),
              callbacks=callbacks_list, verbose=1)

    # Optionally, evaluate the model on test set
    test_loss = model.evaluate([X_notes_test, X_offsets_test, X_durations_test], [y_notes_test, y_offsets_test, y_durations_test], verbose=1)
    print(f"Test Loss: {test_loss}")

def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []
    offsets = []
    durations = []

    for file in glob.glob("/content/content/Keras-LSTM-Music-Generator/classical-piano-type0/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try:  # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:  # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        offsetBase = 0
        for element in notes_to_parse:
            isNoteOrChord = False

            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
                isNoteOrChord = True
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))
                isNoteOrChord = True

            if isNoteOrChord:
                offsets.append(str(element.offset - offsetBase))
                durations.append(str(element.duration.quarterLength))
                isNoteOrChord = False
                offsetBase = element.offset

    with open('data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    with open('data/durations', 'wb') as filepath:
        pickle.dump(durations, filepath)

    with open('data/offsets', 'wb') as filepath:
        pickle.dump(offsets, filepath)

    print(durations)
    return notes, offsets, durations

def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

    # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = to_categorical(network_output)

    return (network_input, network_output)

def create_network(network_input_notes, n_vocab_notes, network_input_offsets, n_vocab_offsets, network_input_durations, n_vocab_durations):
    """ Create the LSTM-based network for music generation """
    # Branch of the network that considers notes
    inputNotesLayer = Input(shape=(network_input_notes.shape[1], network_input_notes.shape[2]))
    inputNotes = LSTM(256, input_shape=(network_input_notes.shape[1], network_input_notes.shape[2]), return_sequences=True)(inputNotesLayer)
    inputNotes = Dropout(0.2)(inputNotes)

    # Branch of the network that considers note offset
    inputOffsetsLayer = Input(shape=(network_input_offsets.shape[1], network_input_offsets.shape[2]))
    inputOffsets = LSTM(256, input_shape=(network_input_offsets.shape[1], network_input_offsets.shape[2]), return_sequences=True)(inputOffsetsLayer)
    inputOffsets = Dropout(0.2)(inputOffsets)

    # Branch of the network that considers note duration
    inputDurationsLayer = Input(shape=(network_input_durations.shape[1], network_input_durations.shape[2]))
    inputDurations = LSTM(256, input_shape=(network_input_durations.shape[1], network_input_durations.shape[2]), return_sequences=True)(inputDurationsLayer)
    inputDurations = Dropout(0.2)(inputDurations)

    # Concatenate the three input networks together into one branch now
    inputs = concatenate([inputNotes, inputOffsets, inputDurations])

    # LSTM to consider everything learnt from the three separate branches
    x = LSTM(512, return_sequences=True)(inputs)
    x = Dropout(0.3)(x)
    x = LSTM(512)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)

    # Time to split into three branches again...

    # Branch of the network that classifies the note
    outputNotes = Dense(128, activation='relu')(x)
    outputNotes = BatchNormalization()(outputNotes)
    outputNotes = Dropout(0.3)(outputNotes)
    outputNotes = Dense(n_vocab_notes, activation='softmax', name="Note")(outputNotes)

    # Branch of the network that classifies the note offset
    outputOffsets = Dense(128, activation='relu')(x)
    outputOffsets = BatchNormalization()(outputOffsets)
    outputOffsets = Dropout(0.3)(outputOffsets)
    outputOffsets = Dense(n_vocab_offsets, activation='softmax', name="Offset")(outputOffsets)

    # Branch of the network that classifies the note duration
    outputDurations = Dense(128, activation='relu')(x)
    outputDurations = BatchNormalization()(outputDurations)
    outputDurations = Dropout(0.3)(outputDurations)
    outputDurations = Dense(n_vocab_durations, activation='softmax', name="Duration")(outputDurations)

    # Tell Keras what our inputs and outputs are
    model = Model(inputs=[inputNotesLayer, inputOffsetsLayer, inputDurationsLayer], outputs=[outputNotes, outputOffsets, outputDurations])

    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

if __name__ == '__main__':
    train_network()


Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/chpn_op7_2.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_wanderer.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/chpn-p24.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/br_im6.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/deb_pass.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/chpn-p11.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/deb_prel.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/deb_clai.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/chpn_op25_e4.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/elise.mid
Parsing /content/content/Keras-LSTM-Music-Generator/classical-piano-type0/beethoven_opus22_2.mid
Parsing /content/content/K

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - Duration_loss: 2.3071 - Note_loss: 5.9550 - Offset_loss: 2.4907 - loss: 10.7528
Epoch 1: val_loss improved from inf to 8.08091, saving model to weights-improvement-01-9.2649-bigger.keras
767/767 ━━━━━━━━━━━━━━━━━━━━ 92s 110ms/step - Duration_loss: 2.3065 - Note_loss: 5.9544 - Offset_loss: 2.4899 - loss: 10.7508 - val_Duration_loss: 1.4259 - val_Note_loss: 5.0841 - val_Offset_loss: 1.5666 - val_loss: 8.0809
Epoch 2/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - Duration_loss: 1.4018 - Note_loss: 5.0837 - Offset_loss: 1.4730 - loss: 7.9585
Epoch 2: val_loss improved from 8.08091 to 7.84382, saving model to weights-improvement-02-7.8600-bigger.keras
767/767 ━━━━━━━━━━━━━━━━━━━━ 81s 106ms/step - Duration_loss: 1.4018 - Note_loss: 5.0837 - Offset_loss: 1.4729 - loss: 7.9583 - val_Duration_loss: 1.3381 - val_Note_loss: 5.1295 - val_Offset_loss: 1.3796 - val_loss: 7.8438
Epoch 3/100
767/767 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - Duratio

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import load_model
import glob
import math

def calculate_ppl(model_path, X_notes_test, X_offsets_test, X_durations_test, y_notes_test, y_offsets_test, y_durations_test):
    """ Load model tốt nhất và tính perplexity (PPL) """
    # Load model tốt nhất
    model = load_model(model_path)

    # Đánh giá model trên tập kiểm tra
    loss = model.evaluate([X_notes_test, X_offsets_test, X_durations_test],
                          [y_notes_test, y_offsets_test, y_durations_test],
                          verbose=1)

    # Vì model có nhiều đầu ra, loss là danh sách [total_loss, loss_notes, loss_offsets, loss_durations]
    total_loss = loss[0]  # Tổng loss của model

    # Tính Perplexity
    ppl = math.exp(total_loss)  # PPL = e^loss
    print(f"Perplexity (PPL) của mô hình: {ppl:.4f}")
    return ppl

# Tìm model tốt nhất đã lưu
model_files = glob.glob("weights-improvement-*.keras")
best_model_path = sorted(model_files, key=lambda x: float(x.split("-")[2]))[0]  # Chọn model có loss nhỏ nhất

# Gọi hàm tính PPL (Thay X_test và y_test bằng dữ liệu thực tế)
ppl_score = calculate_ppl(best_model_path, X_notes_test, X_offsets_test, X_durations_test,
                          y_notes_test, y_offsets_test, y_durations_test)

NameError: name 'X_notes_test' is not defined

  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/ (stored 0%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/beethoven_opus22_3.mid (deflated 78%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/ty_maerz.mid (deflated 52%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_march.mid (deflated 76%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_wanderer.mid (deflated 64%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/ty_januar.mid (deflated 62%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/chpn-p18.mid (deflated 44%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/grieg_elfentanz.mid (deflated 71%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/brahms_opus1_2.mid (deflated 56%)
  adding: content/Keras-LSTM-Music-Generator/classical-piano-type0/schumm-3.mid (deflated 62%)
  adding: content/Keras-LSTM-Mus

In [ ]:
import os

# Thay đổi thư mục làm việc
os.chdir('/content/Keras-LSTM-Music-Generator')

# Kiểm tra lại thư mục hiện tại
print(os.getcwd())

/content/Keras-LSTM-Music-Generator


In [ ]:
!python3 train.py


2025-03-18 04:42:11.093620: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742272931.119592    3908 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742272931.127121    3908 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-18 04:42:11.152140: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Parsing classical-piano-type0/beethoven_opus22_3.mid
Parsing classical-piano-type0/ty_maerz.mid
Parsing classical-piano-type0